In [7]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import hopsworks
import os
import warnings
from functions import util


## GET THE AIR QUALITY FORECAST FOR THE NEXT DAYS

# Here hopsworks key is not a secret 

with open('../../data/hopsworks-api-key.txt', 'r') as file:
     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

## Here it is

# Get the API key from GitHub Secrets
##HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')

#Get AQI API KEY from secrets of hopsworks

proj = hopsworks.login(project="ID2223LAB1KTH")

conn = hopsworks.connection(host="c.app.hopsworks.ai", project=proj, api_key_value=os.environ.get('HOPSWORKS_API_KEY'))
secrets = conn.get_secrets_api()

AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value


country="sweden"
city = "stockholm"
street = "stockholm-hornsgatan-108"
aqicn_url="https://api.waqi.info/feed/@10009"


# Fetch air quality data from the AQICN API
response = requests.get(f"{aqicn_url}/?token={AQI_API_KEY}")

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # 5. Parse the data 
    pm25_data = data.get("data", {}).get("forecast", {}).get("daily", {}).get("pm25", [])
    pm10_data = data.get("data", {}).get("forecast", {}).get("daily", {}).get("pm10", [])


    pm25_df = pd.DataFrame(pm25_data)
    pm10_df = pd.DataFrame(pm10_data)

    pm25_df = pm25_df[['day', 'avg']].rename(columns={'avg': 'pm25'})
    pm10_df = pm10_df[['day', 'avg']].rename(columns={'avg': 'pm10'})
    

    df_forecast_aq = pm25_df.merge(pm10_df, on="day", how="inner").drop(index=0)
    df_forecast_aq[['pm25', 'pm10']] = df_forecast_aq[['pm25', 'pm10']].astype(int)
   
    df_forecast_aq['day'] = pd.to_datetime(df_forecast_aq['day'])
    df_forecast_aq.rename(columns={'day': 'time'}, inplace=True)


## GET THE WEATHER FORECAST FOR THE NEXT DAYS 

# Coordinates of Stockholm
latitude = 59.3293
longitude = 18.0686

# Get the current date and calculate the forecast date range (e.g., next 7 days)
today = datetime.today().date()
forecast_start_date = today - timedelta(days=1)
forecast_end_date = today + timedelta(days=5)  # 7 days from today

df_forecast_weather = util.get_hourly_weather_forecast(city, latitude, longitude)

# Extract the date component (without the time part)
df_forecast_weather['date'] = df_forecast_weather['date'].dt.date

# Group by the new 'day' column and calculate the daily average for each feature
df_forecast_weather = df_forecast_weather.groupby('date').agg({
    'temperature_2m_mean': 'mean',
    'precipitation_sum': 'mean',
    'wind_speed_10m_max': 'mean',
    'wind_direction_10m_dominant': 'mean'
}).reset_index()



print(df_forecast_weather)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170583
Connected. Call `.close()` to terminate connection gracefully.
Coordinates 59.25°N 18.0°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
         date  temperature_2m_mean  precipitation_sum  wind_speed_10m_max  \
0  2024-11-14             4.387500           0.000000           12.027312   
1  2024-11-15             5.318750           0.000000           12.566491   
2  2024-11-16             7.731250           0.000000           22.973274   
3  2024-11-17             4.339583           0.000000           19.935667   
4  2024-11-18             1.372917           0.000000           10.906657   
5  2024-11-19             0.231250           0.058333            8.246800   
6  2024-11-20             3.477083           0.854167           17.302485   
7  2024-11-21             3.045833           0.050000           11.103111   
8  2024-11-22            -0.060417           0.200000           